# Importamos librerias 

In [1]:
import findspark
findspark.init()


In [25]:
#from pyspark.sql import SparkSession #Para crear una sesión de Spark
from pyspark.sql.types import *  
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from collections import Counter #Para contar la frecuencia de los elementos de una lista
import json # Datos en formato JSON, pues los datos de BICIMAD vienen en ese formato
#from pprint import pprint # Para imprimir de manera legible los resultados
import datetime # Para poder trabajar con fechas 
import sys
import statistics

# Creamos sesión en spark

In [3]:
#spark = SparkSession.builder.getOrCreate() #Creamos la sesión
conf = SparkConf()\
        .setAppName('PracticaSpark')
sc = SparkContext(conf = conf)

# Abrimos el archivo correspondiente a junio de 2022


In [4]:
import json
with open(r"C:/Users/j4gon/Downloads/junio2019.json", encoding='latin1') as f:
    data = f.readlines()
     

rdd = sc.textFile(r"C:/Users/j4gon/Downloads/junio2019.json")

In [7]:
def FiletoDic(line):

  data=json.loads(line)
  return data

In [8]:
FiletoDic(rdd.first())

{'_id': {'$oid': '5cf83b752f3843a016be4e2f'},
 'user_day_code': 'e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b',
 'idplug_base': 21,
 'user_type': 1,
 'idunplug_base': 8,
 'travel_time': 219,
 'idunplug_station': 90,
 'ageRange': 0,
 'idplug_station': 66,
 'unplug_hourTime': {'$date': '2019-06-01T00:00:00.000+0200'},
 'zip_code': ''}

# Obtenemos las estaciones de salida y llegada para cada usuario

In [11]:
def salidas(line):
    data    = FiletoDic(line)
    usuario = data['user_day_code']
    salida  = data['idunplug_station'] 
    return usuario,salida

In [12]:
salidas(rdd.first())

('e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b', 90)

In [16]:
def llegadas(line):
    data    = FiletoDic(line)
    usuario = data['user_day_code']
    llegada  = data['idplug_station'] 
    return usuario,llegada

In [17]:
llegadas(rdd.first())

('e4d55deb9ac172a8d8f5f0a32599815bd51b7c8760d67e42b11adf7c0829341b', 66)

# Ahora comprobemos aquellos usuario realizan un camino

Un usuario realiza un camino si la estación de salida y de llegada coinciden

In [52]:
def camino(line):
    data = FiletoDic(line)
    tiempo = data['travel_time']
    if salidas(line) == llegadas(line): 
        return (True, tiempo)
    else:
        return False

In [53]:
camino(rdd.first())

False

In [68]:
usuarios_camino = rdd.map(camino).filter(lambda x: type(x) == tuple).map(lambda x : x[1]).mean()

In [70]:
usuarios_camino

1636.1146547502462

In [72]:
rdd.map(camino).filter(lambda x: type(x) == tuple).collect()

[(True, 357),
 (True, 520),
 (True, 431),
 (True, 921),
 (True, 954),
 (True, 2256),
 (True, 2266),
 (True, 717),
 (True, 567),
 (True, 1272),
 (True, 1830),
 (True, 2596),
 (True, 5477),
 (True, 2857),
 (True, 425),
 (True, 689),
 (True, 155),
 (True, 13),
 (True, 317),
 (True, 1419),
 (True, 23),
 (True, 373),
 (True, 1918),
 (True, 1635),
 (True, 1625),
 (True, 15),
 (True, 494),
 (True, 448),
 (True, 932),
 (True, 1203),
 (True, 1435),
 (True, 741),
 (True, 2160),
 (True, 2276),
 (True, 1872),
 (True, 2805),
 (True, 10928),
 (True, 5601),
 (True, 5926),
 (True, 369),
 (True, 1329),
 (True, 1391),
 (True, 958),
 (True, 1737),
 (True, 768),
 (True, 12),
 (True, 574),
 (True, 4029),
 (True, 4548),
 (True, 867),
 (True, 672),
 (True, 1114),
 (True, 1376),
 (True, 18),
 (True, 1143),
 (True, 3682),
 (True, 161),
 (True, 629),
 (True, 653),
 (True, 1272),
 (True, 4800),
 (True, 9037),
 (True, 16539),
 (True, 141),
 (True, 1554),
 (True, 8),
 (True, 11),
 (True, 13),
 (True, 7),
 (True, 1